In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
import tensorflow as tf
import seaborn as sns
from matplotlib.pylab import rcParams
from sklearn import metrics
from sklearn.ensemble import RandomForestRegressor
from sklearn.cross_decomposition import PLSRegression 
from lightgbm import LGBMRegressor
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

%matplotlib inline

In [3]:
df_1 = pd.read_csv('user1_data.csv')
df_2 = pd.read_csv('user2_data.csv')

In [4]:
df_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19341 entries, 0 to 19340
Data columns (total 8 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   ttime   19341 non-null  object 
 1   pm1     19341 non-null  float64
 2   pm2     19341 non-null  float64
 3   pm3     19341 non-null  float64
 4   am      19341 non-null  float64
 5   sm      19341 non-null  float64
 6   st      19341 non-null  float64
 7   lum     19341 non-null  float64
dtypes: float64(7), object(1)
memory usage: 1.2+ MB


In [5]:
df_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20166 entries, 0 to 20165
Data columns (total 10 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   ttime   20166 non-null  object 
 1   pm1     20166 non-null  float64
 2   pm2     20166 non-null  float64
 3   pm3     20166 non-null  float64
 4   am      20166 non-null  float64
 5   sm      20166 non-null  float64
 6   lum     20166 non-null  float64
 7   temp    20166 non-null  float64
 8   humd    20166 non-null  float64
 9   pres    20166 non-null  float64
dtypes: float64(9), object(1)
memory usage: 1.5+ MB


In [6]:
# sns.heatmap(df_1.corr())

In [7]:
df_1.head()

,ttime,pm1,pm2,pm3,am,sm,st,lum
0,2022-07-18 15:42:55,3.63,0.0,0.0,3.2,7813.0,25.65,15001.0
1,2022-07-18 15:47:55,4.19,0.0,0.0,4.8,7692.0,25.66,13863.0
2,2022-07-18 15:52:55,4.74,0.0,0.0,4.0,7813.0,25.67,13788.0
3,2022-07-18 15:57:55,5.30,0.0,0.0,4.8,7813.0,25.68,14383.0
4,2022-07-18 16:02:55,0.55,5.3,5.3,3.2,7813.0,25.69,14621.0


In [8]:
## plots
# sns.scatterplot(df_2['sm'])
# sns.scatterplot(df_1['sm'])

In [9]:
# df_1.drop('ttime',axis=1,inplace=True)
df_1.columns

Index(['ttime', 'pm1', 'pm2', 'pm3', 'am', 'sm', 'st', 'lum'], dtype='object')

In [68]:
X = df_1.drop(['sm','ttime'],axis=1)
y = df_1["sm"]

In [69]:
# Linear Regression
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=101)
from sklearn.linear_model import LinearRegression
lm = LinearRegression()
lm.fit(X_train,y_train)
predictions = lm.predict(X_test)

## Plots
# plt.scatter(y_test,predictions)

#  Evaluation Metrics
from sklearn import metrics
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, predictions))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, predictions)))


Mean Absolute Error: 1746.1936176175564
Root Mean Squared Error: 1931.6974186003265


In [383]:
# LGBM Regressor

from lightgbm import LGBMRegressor

horizon = 25
    
X_train, X_test = X.iloc[:-horizon,:], X.iloc[-horizon:,:]
y_train, y_test = y.iloc[:-horizon], y.iloc[-horizon:]


model = LGBMRegressor(random_state=42)
model.fit(X_train,y_train)
predictions=model.predict(X_test)

print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, predictions))
print('Mean Squared Error:', metrics.mean_squared_error(y_test, predictions,squared=False))

Mean Absolute Error: 12.180156944162222
Mean Squared Error: 15.210097486008056


In [316]:
# model.predict()
# 
# 
abs(model.predict(X)-y)


0        2538.963190
1        2375.408312
2        2496.408312
3        2496.408312
4         210.806313
            ...     
19336       5.952427
19337       4.952427
19338       5.952427
19339       5.952427
19340       4.937987
Name: sm, Length: 19341, dtype: float64

In [317]:
## PLSRegression

from sklearn.cross_decomposition import PLSRegression 
model = PLSRegression(n_components=4)

horizon = 25

X_train, X_test = X.iloc[:-horizon,:], X.iloc[-horizon:,:]
y_train, y_test = y.iloc[:-horizon], y.iloc[-horizon:]

model.fit(X_train,y_train)
predictions=model.predict(X_test)

print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, predictions))
print('Mean Squared Error:', metrics.mean_squared_error(y_test, predictions,squared=False))

Mean Absolute Error: 1414.0793595542036
Mean Squared Error: 1425.5525297354152


In [318]:
# Random Forest

from sklearn.model_selection import train_test_split


X = df_1.drop(['sm','ttime'], axis=1)
y = df_1['sm']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=101)

from sklearn.ensemble import RandomForestRegressor
from sklearn import metrics

model = RandomForestRegressor(max_depth=2, random_state=0)
model.fit(X, y)
predictions = model.predict(X_test)

print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, predictions))
print('Mean Squared Error:', metrics.mean_squared_error(y_test, predictions,squared=False))


Mean Absolute Error: 384.6741005487055
Mean Squared Error: 647.3186565621697


In [34]:
# Neural Network
# import tensorflow as tf
# print("Importing ...")
# print("Imported")

# Normalize the data

X = df_2.drop(['sm','ttime'], axis=1)
y = df_2['sm']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=101)


x_train = tf.keras.utils.normalize(X_train , axis=1)
x_test = tf.keras.utils.normalize(X_test , axis=1)

model = tf.keras.models.Sequential()

model.add(tf.keras.layers.Dense(15,activation='relu'))

model.add(tf.keras.layers.Dense(30,activation='relu'))

model.add(tf.keras.layers.Dense(25,activation='relu'))

model.add(tf.keras.layers.Dense(1,activation='relu'))

model.compile(loss='mse',optimizer='adam')

model.fit(x_train,y_train,epochs = 250)

model.save('soil.model')

Epoch 1/250
442/442 [==============================] - 1s 1ms/step - loss: 28140984.0000
Epoch 2/250
442/442 [==============================] - 1s 1ms/step - loss: 28140974.0000
Epoch 3/250
442/442 [==============================] - 1s 1ms/step - loss: 28140978.0000
Epoch 4/250
442/442 [==============================] - 1s 1ms/step - loss: 28140970.0000
Epoch 5/250
442/442 [==============================] - 1s 1ms/step - loss: 28140984.0000
Epoch 6/250
442/442 [==============================] - 1s 1ms/step - loss: 28140988.0000
Epoch 7/250
442/442 [==============================] - 1s 1ms/step - loss: 28140988.0000
Epoch 8/250
442/442 [==============================] - 1s 1ms/step - loss: 28140966.0000
Epoch 9/250
442/442 [==============================] - 1s 1ms/step - loss: 28140984.0000
Epoch 10/250
442/442 [==============================] - 1s 1ms/step - loss: 28140968.0000
Epoch 11/250
442/442 [==============================] - 1s 1ms/step - loss: 28140966.0000
Epoch 12/250
442/44

442/442 [==============================] - 1s 1ms/step - loss: 28140980.0000
Epoch 93/250
442/442 [==============================] - 1s 1ms/step - loss: 28140992.0000
Epoch 94/250
442/442 [==============================] - 1s 1ms/step - loss: 28140976.0000
Epoch 95/250
442/442 [==============================] - 1s 1ms/step - loss: 28140982.0000
Epoch 96/250
442/442 [==============================] - 1s 2ms/step - loss: 28140978.0000
Epoch 97/250
442/442 [==============================] - 1s 2ms/step - loss: 28140982.0000
Epoch 98/250
442/442 [==============================] - 1s 2ms/step - loss: 28140962.0000
Epoch 99/250
442/442 [==============================] - 1s 2ms/step - loss: 28140970.0000
Epoch 100/250
442/442 [==============================] - 1s 1ms/step - loss: 28140984.0000
Epoch 101/250
442/442 [==============================] - 1s 2ms/step - loss: 28140976.0000
Epoch 102/250
442/442 [==============================] - 1s 2ms/step - loss: 28140970.0000
Epoch 103/250
442/44

In [319]:
model = tf.keras.models.load_model('soil.model')
# loss, accuracy = model.evaluate(X_test,y_test)

In [320]:
model.evaluate(x_test,y_test)

ValueError: Data cardinality is ambiguous:
  x sizes: 5041
  y sizes: 3869
Make sure all arrays contain the same number of samples.

In [113]:
# # LSTM 
# from sklearn.preprocessing import MinMaxScaler
# from keras.models import Sequential
# from keras.layers import Dense, LSTM




# features = df_2.drop('ttime',axis=1)
# # print(features.shape)

# training_dataset_length = math.ceil(len(features) * .75)
# # print(training_dataset_length)

# #Scale all of the data to be values between 0 and 1
# scaler = MinMaxScaler(feature_range=(0, 1))
# scaled_data = scaler.fit_transform(features)

# train_data = scaled_data[0:training_dataset_length , : ]

# #Splitting the data
# x_train=[]
# y_train = []
# for i in range(10, len(train_data)):
#     x_train.append(train_data[i-1:i,4])
#     y_train.append(train_data[i,4])
    
# #Convert to numpy arrays
# x_train, y_train = np.array(x_train), np.array(y_train)
# #Reshape the data into 3-D array
# x_train = np.reshape(x_train, (x_train.shape[0],x_train.shape[1],1))    

# from keras.layers import Dropout
# # Initialising the RNN
# model = Sequential()
# model.add(LSTM(units = 50, return_sequences = True, input_shape = (x_train.shape[1], 1)))
# model.add(Dropout(0.2))
# # Adding a second LSTM layer and Dropout layer
# model.add(LSTM(units = 50, return_sequences = True))
# model.add(Dropout(0.2))
# # Adding a third LSTM layer and Dropout layer
# model.add(LSTM(units = 50, return_sequences = True))
# model.add(Dropout(0.2))
# # Adding a fourth LSTM layer and and Dropout layer
# model.add(LSTM(units = 50))
# model.add(Dropout(0.2))
# # Adding the output layer
# # For Full connection layer we use dense
# # As the output is 1D so we use unit=1
# model.add(Dense(units = 1))


# #compile and fit the model on 30 epochs
# model.compile(optimizer = 'adam', loss = 'mean_squared_error')
# model.fit(x_train, y_train, epochs =30, batch_size = 1000)

# # model.save("LSTM.model")




In [114]:
# #Test data set
# # test_data = scaled_data[training_dataset_length - 10: , : ]
# #splitting the x_test and y_test data sets
# x_test = []
# y_test = df_2.drop('ttime',axis=1)['sm'].to_numpy()[training_dataset_length : ]
# for i in range(10,len(test_data)):
#     x_test.append(test_data[i-1:i,4])
# #Convert x_test to a numpy array
# x_test = np.array(x_test)
# #Reshape the data into 3-D array
# x_test = np.reshape(x_test, (x_test.shape[0],x_test.shape[1],1))


In [115]:

# #check predicted values
# predictions = model.predict(x_test)
# # Undo scaling
# # predictions = scaler.inverse_transform(predictions)
# # predictions = predictions.reshape((predictions.shape[0], -1))
# # #Calculate RMSE score

# # rmse=np.sqrt(np.mean((([:,4]- y_test[:,4])**2)))



# # rmse

In [55]:
predictions.shape


(5041, 1)

In [56]:
y_test

array([469., 468., 468., ..., 327., 327., 328.])

In [57]:
predictions = scaler.inverse_transform(predictions)
predictions = predictions.reshape((predictions.shape[0], -1))
print('Mean Squared Error:', metrics.mean_squared_error(y_test, predictions,squared=False))


Mean Squared Error: 368.7440618480433


In [66]:
predictions[0]


array([0.07863206], dtype=float32)

In [141]:
 df_1.columns[1:]

# [0,1,2,3,5,6]

Index(['pm1', 'pm2', 'pm3', 'am', 'sm', 'st', 'lum'], dtype='object')

In [360]:
# LSTM 


from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, LSTM

input_fetaure_index=[2,4,5]
no_of_features= len(input_fetaure_index)
target_index = 4
in_days=10

In [361]:
df = df_1.drop('ttime',axis=1)

fraction = math.ceil(len(df) * .75)

scaler = MinMaxScaler(feature_range=(0, 1))

scaled_data = scaler.fit_transform(df)

train_data = scaled_data[0:fraction , : ]

In [362]:
#Splitting the data
x_train=[]
y_train = []
for i in range(in_days, len(train_data)):
    x_train.append(train_data[i-in_days:i,input_fetaure_index])
    y_train.append(train_data[i,target_index])

In [363]:
x_train

[array([[0.        , 0.98379297, 0.22580645],
        [0.        , 0.96797804, 0.22720898],
        [0.        , 0.98379297, 0.2286115 ],
        [0.        , 0.98379297, 0.23001403],
        [0.03449173, 0.98379297, 0.23141655],
        [0.03449173, 0.98379297, 0.2342216 ],
        [0.03449173, 0.98379297, 0.23562412],
        [0.03449173, 0.98379297, 0.23702665],
        [0.03449173, 0.98379297, 0.23842917],
        [0.03449173, 0.96797804, 0.2398317 ]]),
 array([[0.        , 0.96797804, 0.22720898],
        [0.        , 0.98379297, 0.2286115 ],
        [0.        , 0.98379297, 0.23001403],
        [0.03449173, 0.98379297, 0.23141655],
        [0.03449173, 0.98379297, 0.2342216 ],
        [0.03449173, 0.98379297, 0.23562412],
        [0.03449173, 0.98379297, 0.23702665],
        [0.03449173, 0.98379297, 0.23842917],
        [0.03449173, 0.96797804, 0.2398317 ],
        [0.03449173, 0.96797804, 0.24123422]]),
 array([[0.        , 0.98379297, 0.2286115 ],
        [0.        , 0.9837929

In [364]:
#Convert to numpy arrays
x_train, y_train = np.array(x_train), np.array(y_train)
#Reshape the data into 3-D array
# x_train = np.reshape(x_train, (x_train.shape[0],x_train.shape[1],no_of_features))    

In [365]:
x_train

array([[[0.        , 0.98379297, 0.22580645],
        [0.        , 0.96797804, 0.22720898],
        [0.        , 0.98379297, 0.2286115 ],
        ...,
        [0.03449173, 0.98379297, 0.23702665],
        [0.03449173, 0.98379297, 0.23842917],
        [0.03449173, 0.96797804, 0.2398317 ]],

       [[0.        , 0.96797804, 0.22720898],
        [0.        , 0.98379297, 0.2286115 ],
        [0.        , 0.98379297, 0.23001403],
        ...,
        [0.03449173, 0.98379297, 0.23842917],
        [0.03449173, 0.96797804, 0.2398317 ],
        [0.03449173, 0.96797804, 0.24123422]],

       [[0.        , 0.98379297, 0.2286115 ],
        [0.        , 0.98379297, 0.23001403],
        [0.03449173, 0.98379297, 0.23141655],
        ...,
        [0.03449173, 0.96797804, 0.2398317 ],
        [0.03449173, 0.96797804, 0.24123422],
        [0.03449173, 0.98379297, 0.24403927]],

       ...,

       [[0.25087856, 0.03319827, 0.35063114],
        [0.25087856, 0.03332898, 0.34642356],
        [0.25087856, 0

In [366]:
x_train

array([[[0.        , 0.98379297, 0.22580645],
        [0.        , 0.96797804, 0.22720898],
        [0.        , 0.98379297, 0.2286115 ],
        ...,
        [0.03449173, 0.98379297, 0.23702665],
        [0.03449173, 0.98379297, 0.23842917],
        [0.03449173, 0.96797804, 0.2398317 ]],

       [[0.        , 0.96797804, 0.22720898],
        [0.        , 0.98379297, 0.2286115 ],
        [0.        , 0.98379297, 0.23001403],
        ...,
        [0.03449173, 0.98379297, 0.23842917],
        [0.03449173, 0.96797804, 0.2398317 ],
        [0.03449173, 0.96797804, 0.24123422]],

       [[0.        , 0.98379297, 0.2286115 ],
        [0.        , 0.98379297, 0.23001403],
        [0.03449173, 0.98379297, 0.23141655],
        ...,
        [0.03449173, 0.96797804, 0.2398317 ],
        [0.03449173, 0.96797804, 0.24123422],
        [0.03449173, 0.98379297, 0.24403927]],

       ...,

       [[0.25087856, 0.03319827, 0.35063114],
        [0.25087856, 0.03332898, 0.34642356],
        [0.25087856, 0

In [367]:
from keras.layers import Dropout

model = Sequential()
model.add(LSTM(units = 50, return_sequences = True, input_shape = (x_train.shape[1], no_of_features)))
model.add(Dropout(0.2))

model.add(LSTM(units = 70, return_sequences = True, input_shape = (x_train.shape[1], no_of_features)))
model.add(Dropout(0.2))

model.add(LSTM(units = 50, return_sequences = True, input_shape = (x_train.shape[1], no_of_features)))
model.add(Dropout(0.2))

model.add(LSTM(units = 50))
model.add(Dropout(0.2))

model.add(Dense(1, activation='relu'))


In [368]:
model.compile(optimizer = 'adam', loss = 'mean_squared_error')

In [369]:
model.fit(x_train, y_train, epochs =5, batch_size = 1000)

Epoch 1/5
15/15 [==============================] - 10s 89ms/step - loss: 0.2120
Epoch 2/5
15/15 [==============================] - 1s 87ms/step - loss: 0.0348
Epoch 3/5
15/15 [==============================] - 1s 93ms/step - loss: 0.0201
Epoch 4/5
15/15 [==============================] - 2s 105ms/step - loss: 0.0109
Epoch 5/5
15/15 [==============================] - 2s 101ms/step - loss: 0.0085


In [370]:
#Test data set
test_data = scaled_data[fraction - in_days: , : ]
#splitting the x_test and y_test data sets
x_test = []
y_test = df_1.drop('ttime',axis=1)['sm'].to_numpy()[fraction : ]
for i in range(in_days,len(test_data)):
    x_test.append(test_data[i-in_days:i,input_fetaure_index])
# #Convert x_test to a numpy array
x_test = np.array(x_test)
# #Reshape the data into 3-D array
x_test = np.reshape(x_test, (x_test.shape[0],x_test.shape[1],no_of_features))
# print(x_test.shape, y_test.shape)


In [371]:
x_test.shape

(4835, 10, 3)

In [372]:

#check predicted values
predictions = model.predict(x_test)
# # Undo scaling
temp = df_1.drop('ttime', axis=1).values
predictions = np.concatenate((temp[:len(predictions), :target_index],predictions, temp[:len(predictions), target_index+1:]), axis=1)
predictions = scaler.inverse_transform(predictions)
predictions = predictions[:, target_index]


152/152 [==============================] - 3s 6ms/step


In [373]:
print('Mean Squared Error:', metrics.mean_squared_error(y_test, predictions,squared=False))


Mean Squared Error: 263.07965516193406


In [374]:
predictions

array([683.30097525, 689.32320426, 693.67400749, ..., 286.        ,
       286.        , 286.        ])

In [375]:
y_test

array([546., 546., 545., ..., 291., 291., 292.])